In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import base64
import os
import dash_leaflet as dl
import plotly.express as px
from dash.dependencies import Input, Output
from dash import html
from dash import dash_table
from dash import dcc


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#importing Animal Shelter
from AnimalShelterDAL import AnimalShelterDAL



###########################
# Data Manipulation / Model
###########################

#initialized object for AnimalShelterDAL here using login credentials
#and by getting the port from os so that the DAL can be used to retrieve all
#from the AAC database 'animals' collection
username = "aacuser"
password = "password123"
port = os.environ["MD_PORT"]
shelter = AnimalShelterDAL(username, password, port)
shelter.connect('AAC')

# class read method must support return of cursor object and accept projection json input
#dataframe here to read all from the 'animals' collection 
df = pd.DataFrame.from_records(shelter.read('animals', {}))

#importing the company logo to use in the UI
#this will encode image in base 64 and then in the html it will be un-encoded
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#########################
# Dashboard Layout / View
#########################
#using Dash() here because Jupyter Dash would not load for me,
#it would just perpetually give me a "loading..." message then not load
#this will initialize the Dash and ability to use the framework for the dashboards
app = dash.Dash('Animal Shelter Dashboard View')

#this app layout will be the page layout and dash framework html type code to display everything on the page
#this will display the page title, the Grazioso logo, the animal data table, and the dashboard map
app.layout = html.Div([
    #adding the company logo into the display using the <img> tag to display on screen
    #this will shrink it a bit and center it on the page using the style formatting
    html.Img(id='logo', src='data:image/png;base64,{}'.format(encoded_image.decode()), alt='logo',
            style={'display':'block', 'margin-left':'auto', 'margin-right':
                  'auto', 'width': '30%'}),
    
    #adding in buttons for filtering between water rescue, mountain or wilderness rescue,
    #disaster or individual tracking, and reset button to show all
    html.Hr(),
    html.Center(html.B(html.H2('Animal Data Table'))),
    html.Hr(),
    #made this a flexbox and added some styling to how the buttons are aligned on page
    html.Div(className='row', style={'display':'flex', 'justify-content': 'center', 'align-items': 'center',
                                    'gap': '15px'},
             #there are 4 buttons here, one for each rescue type and one for "see all"
            children=[html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                     html.Button(id='submit-button-two', n_clicks=0, children='Mountain/Wilderness Rescue'),
                     html.Button(id='submit-button-three', n_clicks=0, children='Disaster/Individual Tracking'),
                     html.Button(id='submit-button-four', n_clicks=0, children='See All')]),
    
    #adding in the data table here and giving it attributes to make it able to be filtered by columns
    #this data table will utilize a function to retrieve all records from the AAC database and then will display
    #these records in a table format, can be filtered using the above cat and dog buttons
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #added filtering options as well as limited page size so only 10 results show at a time,
        #which makes the UI look a bit cleaner and the dashboard less hidden away at the bottom of the table
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        page_size=10
    ),
    html.Br(),
    #adding in the geolocation map here
    #adding in a pie chart here as well
    #this uses flex box to display these in the same row on the screen
    html.Hr(),
    html.Center(html.B(html.H2('Animal Geolocation Chart And Pie Chart'))),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[   
        html.Div(
            #this first one is for the pie chart, corresponds to the graph function below
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            #second this is for the geolocation map, corresponds to map function below
            id='map-id',
            className='col s12 m6',
            )
        ]),
    #added my unique identifier at the bottom of the UI instead of at the top, since the logo
    #should be the main focal point, this is just the "end credits" of who wrote the code for the site
    html.Center('Developer: Erin Walter CS-340 Dashboard')
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
#so this highlights the cell on the data table and is used by referencing the above
#table via "datatable-id" id
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#this method will be utilized to return a geolocation map of animals from the data table
#it is referenced in a div above using "map-id" and gets the location_lat and location_long data
#from items on the table and then displays them using pins on the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)

    lat = dff.loc[1, 'location_lat']
    lon = dff.loc[1, 'location_long']
    
    # generate the map using these lat and lon values
    #map is centered on the location of austin AAC animal center
    #with a marker at lat/lon
    return [
        dl.Map(style={'width': '1000px', 'height': '500px', 'display':'block', 'margin-left':'auto', 'margin-right':
                  'auto'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.loc[1,'breed']),
                dl.Popup([
                    html.H2("Animal Name"),
                    html.P(dff.loc[1,'name'])
                ])
            ])
        ])
    ]

#this is the function that allows for the clicking of the butons to filter the table
#it will read from the database in the DAL layer and pass in the query that needs to be
#sent based on what button was clicked
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
    Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')])
def on_click(waterFilter, mountainFilter, trackingFilter, resetFilter ):
    ctx = dash.callback_context

    df = pd.DataFrame.from_records(shelter.read('animals', {}))
    
    if not ctx.triggered:
        button_id = "not clicked yet"
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    #for water rescue, this will get back all dogs who are intact female,
    #in the breed list below, and between 26-156 weeks old
    if button_id == 'submit-button-one':
        df = pd.DataFrame.from_records(shelter.read('animals', {"animal_type":"Dog", 
                                                                "sex_upon_outcome" :"Intact Female",
                                                                "breed" : {"$in" : ["Labrador Retriever Mix", "Chesapeake Bay Retriever","Newfoundland"]},
                                                               "age_upon_outcome_in_weeks": {"$gte": 26, "$lte" : 156}}))
    #for mountain rescue, will get back intact male dogs between 26-156 weeks
    #who are in that breed array below
    elif button_id == 'submit-button-two':
        df = pd.DataFrame.from_records(shelter.read('animals', {"animal_type":"Dog", 
                                                                "sex_upon_outcome" :"Intact Male",
                                                                "breed" : {"$in" : ["German Shepherd", "Alaskan Malamute","Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                                                               "age_upon_outcome_in_weeks": {"$gte": 26, "$lte" : 156}}))
    #for disaster tracking, will get back intact male dogs 20-300 weeks, in the
    #breed array below 
    elif button_id == 'submit-button-three':
        df = pd.DataFrame.from_records(shelter.read('animals', {"animal_type":"Dog", 
                                                                "sex_upon_outcome" :"Intact Male",
                                                                "breed" : {"$in" : ["German Shepherd", "Doberman Pinscher","Golden Retriever", "Bloodhound", "Rottweiler"]},
                                                               "age_upon_outcome_in_weeks": {"$gte": 20, "$lte" : 300}}))
    #this is the "reset" button which will show all types so gets an
    #empty array put in to get all animals back
    elif button_id == 'submit-button-four':
        df = pd.DataFrame.from_records(shelter.read('animals', {}))
    
    return df.to_dict('records')

#this is the function that will display the pie chart on the screen
#its inputs are the viewdata from the table, and it will take this info
#and then turn it into a pie chart showing percent breakdown of breeds
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):

    #this will get the data based on the viewdata frame
    #so based on what is currently filtered
    df = pd.DataFrame.from_dict(viewData)
    
    #having it label this by breed to show the breakdown
    #of what percentages of what breeds are in each category
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed')
        )    
    ]


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "Animal Shelter Dashboard View" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2022 16:04:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:04:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:04:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:04:54] "GET /_favicon.ico?v=2.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:04:54] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 16:04:54] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.ha

127.0.0.1 - - [14/Aug/2022 16:04:54] "

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/erinwalter_snhu/.local/lib/python3.6/site-packages/dash/dash.py

POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Aug/2022 16:04:54] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Aug/2022 16:04:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:04:55] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/erinwalter_snhu/.local/lib/python3.6/site-packages/dash/dash.py

127.0.0.1 - - [14/Aug/2022 16:04:56] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [14/Aug/2022 16:05:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:05:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:05:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:05:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:05:16] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 16:05:16] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Aug/2022 16:06:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:06:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:06:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:06:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2022 16:06:34] "